In [2]:
# We'll be using python's request library to interact with elasticsearch's REST API 

from requests import get, post, put, delete

### Creating & Checing Indices

In [3]:
url = 'http://localhost:9200/courses?&pretty'
resp = put(url)

In [4]:
resp.json()

{u'acknowledged': True, u'index': u'courses', u'shards_acknowledged': True}

In [5]:
# Checking the index we have just created in above step.
url = 'http://localhost:9200/_cat/indices?v&pretty'
resp = get(url)
resp.text

u'health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size\nyellow open   courses Ubpbhk4-Q2SfTxautoqfBg   1   1          0            0       230b           230b\n'

### Creating a Document & Getting it back via Id

In [8]:
data = {
    'name': 'Python3',
    'creditHours': 3,
    'totalLectures': 30
}

In [9]:
url = 'http://localhost:9200/courses/programming/1?pretty'
resp = put(url, json=data)

In [10]:
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 0,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'programming',
 u'_version': 1,
 u'result': u'created'}

In [11]:
data = {
    'name': 'C++',
    'creditHours': 4,
    'totalLectures': 10
}
url = 'http://localhost:9200/courses/programming/2?pretty'
resp = put(url, json=data)

In [12]:
resp.json()

{u'_id': u'2',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 1,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'programming',
 u'_version': 1,
 u'result': u'created'}

In [13]:
# Getting the document via ID

url = 'http://localhost:9200/courses/programming/1?pretty' # &_source=false, &_source=name, &_source=name,totalLectures
resp = get(url)
resp

<Response [200]>

In [14]:
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 0,
 u'_source': {u'creditHours': 3, u'name': u'Python3', u'totalLectures': 30},
 u'_type': u'programming',
 u'_version': 1,
 u'found': True}

### Updating Documner

In [15]:
# Updating whole document via pur request
data = {
    'name': 'Python3',
    'creditHours': 3,
    'totalLectures': 30
}

url = 'http://localhost:9200/courses/programming/1?pretty'
resp = put(url, json=data)

resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 2,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'programming',
 u'_version': 2,
 u'result': u'updated'}

In [16]:
# Updating a document via POST request

url = 'http://localhost:9200/courses/programming/1/_update?pretty'
data = {
    'doc': {
        'totalLectures': 30
    }
}

resp = post(url, json=data)
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 2,
 u'_shards': {u'failed': 0, u'successful': 0, u'total': 0},
 u'_type': u'programming',
 u'_version': 2,
 u'result': u'noop'}

In [17]:
# Updating the document via script
url = 'http://localhost:9200/courses/programming/1/_update?pretty'
data = {
    'script': 'ctx._source.totalLectures += 2'
}
resp = post(url, json=data)
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 3,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'programming',
 u'_version': 3,
 u'result': u'updated'}

In [18]:
url = 'http://localhost:9200/courses/programming/1?pretty' # &_source=false, &_source=name, &_source=name,totalLectures
resp = get(url)
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 3,
 u'_source': {u'creditHours': 3, u'name': u'Python3', u'totalLectures': 32},
 u'_type': u'programming',
 u'_version': 3,
 u'found': True}

### Deleting a document

In [19]:
url = 'http://localhost:9200/courses/programming/1?pretty'
resp = delete(url)
resp.json()

{u'_id': u'1',
 u'_index': u'courses',
 u'_primary_term': 1,
 u'_seq_no': 4,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'programming',
 u'_version': 4,
 u'result': u'deleted'}

### Deleting an entire index

In [61]:
# If you delete the entire index, you'll loose your data. Be careful!
url = 'http://localhost:9200/courses?pretty'
resp = delete(url)
resp.json()

{u'acknowledged': True}

### Getting multiple documents using mGet

In [20]:
url = 'http://localhost:9200/_mget?pretty'
data = {
    'docs': [
        {
            '_index': 'courses',
            '_type': 'programming',
            '_id': 1
        },
        {
            '_index': 'courses',
            '_type': 'programming',
            '_id': 2
        },
    ]
}
resp = post(url, json=data)

In [21]:
resp.json()

{u'docs': [{u'_id': u'1',
   u'_index': u'courses',
   u'_type': u'programming',
   u'found': False},
  {u'_id': u'2',
   u'_index': u'courses',
   u'_primary_term': 1,
   u'_seq_no': 1,
   u'_source': {u'creditHours': 4, u'name': u'C++', u'totalLectures': 10},
   u'_type': u'programming',
   u'_version': 1,
   u'found': True}]}

In [22]:
# The above request can used as follows
url = 'http://localhost:9200/courses/programming/_mget?pretty'
data = {
    'docs': [
        {
            '_id':2
        },
        {
            '_id': 2
        },
    ]
}
resp = post(url, json=data)
resp.json()

{u'docs': [{u'_id': u'2',
   u'_index': u'courses',
   u'_primary_term': 1,
   u'_seq_no': 1,
   u'_source': {u'creditHours': 4, u'name': u'C++', u'totalLectures': 10},
   u'_type': u'programming',
   u'_version': 1,
   u'found': True},
  {u'_id': u'2',
   u'_index': u'courses',
   u'_primary_term': 1,
   u'_seq_no': 1,
   u'_source': {u'creditHours': 4, u'name': u'C++', u'totalLectures': 10},
   u'_type': u'programming',
   u'_version': 1,
   u'found': True}]}

### Bulk loading via json file

In [24]:
# Loading file into elastic index students

url = 'http://localhost:9200/students/personal/_bulk?pretty&refresh'
data = open('students_full.json', 'rb').read()
headers = {
    "Content-Type":"application/x-ndjson",
}
resp = post(url,data=data,headers=headers)
resp

<Response [200]>